In [9]:
import scipy as sp
import scipy.sparse as sps
import numpy as np
import torch
import xitorch
from xitorch import linalg



EJ = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
EL = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
ECs = torch.tensor(20, requires_grad=True, dtype=torch.double)
EC = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
dEj = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
dCj = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
ECj =  torch.tensor(0.2, requires_grad=True, dtype=torch.double)
flux = torch.tensor(0.5, requires_grad=True, dtype=torch.double)
ng = 0.1
ncut = 30
truncated_dim = 10
pt_count = 10
min_val = -19
max_val = 19
hamiltonian_creation = 'auto_H'


phi_ext = 0.5
varphi_ext =0.5

Nphi = 100
Ntheta = 100

eye_Nphi = torch.eye(Nphi)
eye_Ntheta = torch.eye(Ntheta)

partial_phi_fd = torch.kron(eye_Ntheta, torch.tensor(sps.diags([-1, 1, 1], [0, 1, -Nphi+1], shape=(Nphi, Nphi)).todense()))
partial_phi_bk = torch.kron(eye_Ntheta, torch.tensor(sps.diags([1, -1, -1], [0, -1, Nphi-1], shape=(Nphi, Nphi)).todense()))
partial_theta_fd = torch.kron(eye_Nphi, torch.tensor(sps.diags([-1, 1, 1], [0, 1, -Ntheta+1], shape=(Ntheta, Ntheta)).todense()))
partial_theta_bk =torch.kron(eye_Nphi, torch.tensor(sps.diags([1, -1, -1], [0, -1, Ntheta-1], shape=(Ntheta, Ntheta)).todense()))


phi = np.linspace(0, 2 * np.pi, Nphi)
cos_phi = np.cos(phi)
cos_phi_m = np.diag(cos_phi)
sin_phi_adj = np.sin(phi-phi_ext/2)
sin_phi_adj_m = np.diag(sin_phi_adj)
phi_m = np.diag(phi)
_cos_phi = torch.kron(torch.tensor(cos_phi_m), torch.tensor(eye_Nphi) )
_phi = torch.kron(torch.tensor(phi_m), torch.tensor(eye_Nphi) )
_sin_phi_adj_m  = torch.kron(torch.tensor(sin_phi_adj_m), torch.tensor(eye_Nphi))

theta = np.linspace(0, 2 * np.pi, Ntheta)
cos_theta_adj = np.cos(theta-varphi_ext/2)
sin_theta = np.sin(theta)
cos_theta_adj_m = np.diag(cos_theta_adj)
sin_theta_m = np.diag(sin_theta)
_cos_theta_adj_m = torch.kron(torch.tensor(cos_theta_adj_m), torch.tensor(eye_Ntheta))
_sin_theta_m = torch.kron(torch.tensor(sin_theta_m), torch.tensor(eye_Ntheta))


I = torch.kron(torch.tensor(eye_Nphi), torch.tensor(eye_Ntheta))

H = -2 * ECj * (partial_phi_fd * partial_phi_bk) \
    + 2 * ECs * (-1* partial_theta_fd**2 +ng**2*I-2*ng*partial_theta_fd)\
    + 2 * ECs * dCj * partial_phi_fd * partial_theta_fd \
    - 2 * EJ * _cos_phi * _cos_theta_adj_m \
    + EL * _phi ** 2 \
    + 2 * EJ * I  \
    + EJ * dEj * _sin_theta_m * _sin_phi_adj_m


/var/folders/tf/s0sdd8z90kj1wtwzbjj9715r0000gn/T/ipykernel_87426/3397424474.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _cos_phi = torch.kron(torch.tensor(cos_phi_m), torch.tensor(eye_Nphi) )
/var/folders/tf/s0sdd8z90kj1wtwzbjj9715r0000gn/T/ipykernel_87426/3397424474.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _phi = torch.kron(torch.tensor(phi_m), torch.tensor(eye_Nphi) )
/var/folders/tf/s0sdd8z90kj1wtwzbjj9715r0000gn/T/ipykernel_87426/3397424474.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _sin_phi_adj_m  = tor

In [12]:
H_h = H + torch.transpose(H, 1,0 )  #NOT SURE IF THIS IS ENTIRLEY TRUE
A =xitorch.LinearOperator.m(H_h)
xitorch.LinearOperator._getparamnames(A, "EJ, EJ")

val, vec = xitorch.linalg.symeig(A, 2, method="davidson", max_niter=100, nguess=None, v_init="randn", max_addition=None, min_eps=1e-06, verbose=False)

dE = val[1] - val[0]
dE.backward()
print(EL.grad)
print(EJ.grad)

tensor(0.0330, dtype=torch.float64)
tensor(0.0162, dtype=torch.float64)
